# Using Widgets

In [1]:
import sys
print(sys.version)
import ipywidgets as widgets
from ipywidgets import HBox, VBox ,Layout
from IPython.display import display
import serial

3.7.2 (default, Dec 29 2018, 06:19:36) 
[GCC 7.3.0]


# Calibrate servos

In [2]:
class armCalibrate:
    def __init__(self, serialPath = '/dev/ttyACM0', baudrate = 115200):
        self.servoMax = [125,125,125,125]
        self.servoMin = [40,40,40,40]
        self.servoZeroPosition = [73,82,75,0]
        self.servoListRaw = self.servoZeroPosition
        self.serial = serial.Serial(serialPath, baudrate=baudrate)
        # setup widgets
        self.setUpWidgets()
    
    def closeSerial(self):
        self.serial.close()
    
    def setUpWidgets(self):
        style = {'description_width': 'initial'}
        self.serialCommandTextWidget = widgets.Text(description = "sent command via USB", style=style)
        self.rawServoValueWidgets = [
            widgets.IntSlider(value = self.servoListRaw[0],
                              min = self.servoMin[0], max = self.servoMax[0], step = 1, 
                              description = "Right Servo Raw",
                              continuous_update=False, style=style),
            widgets.IntSlider(value = self.servoListRaw[1],
                              min = self.servoMin[1], max = self.servoMax[1], step = 1, 
                              description = "Left Servo Raw",
                              continuous_update=False, style=style),
            widgets.IntSlider(value = self.servoListRaw[2],
                              min = self.servoMin[2], max = self.servoMax[2], step = 1, 
                              description = "Base Servo Raw",
                              continuous_update=False, style=style),
            widgets.IntSlider(value = self.servoListRaw[3],
                              min = self.servoMin[3], max = self.servoMax[3], step = 1, 
                              description = "Gripper Servo Raw",
                              continuous_update=False, style=style)
        ]
        for w in self.rawServoValueWidgets:
            w.observe(self.readAllRawWidgets)
        
        l = [w for w in self.rawServoValueWidgets]
        l.append(self.serialCommandTextWidget)
        self.box = VBox(l,layout = Layout(width='100%'))
        display(self.box)
    
    def readAllRawWidgets(self, updateServos = True):
        for i, w in enumerate(self.rawServoValueWidgets):
            self.servoListRaw[i] = w.value
        if (updateServos):
            self.sendServos()
    
    def valuesToString(self, values):
        if (len(values)!=4):
            return None
        return 's'+','.join([str(v) for v in values]) + 'e'
    
    def sendServos(self):
        st = self.valuesToString(self.servoListRaw)
        if (st == None):
            return
        if not self.serial.isOpen():
            self.serial.open()
        self.serial.write(st.encode())
        self.serial.close()
        self.serialCommandTextWidget.value = st

In [3]:
calibrate = armCalibrate()

My setup had the following values:

| Servo| min value | min deg | value for zero deg | max value
|------|------|------|------|------|
|Right|46| -45 |73|121|
|Left|46| -60 |82|98|
|Base|41| -60 |75|123|
|Gripper|47| -95 |100|125|

Then we fill the calibration values in the init of the following class

In [4]:
calibrate.closeSerial()

In [5]:
class armController:
    def __init__(self, serialPath = '/dev/ttyACM0', baudrate = 115200):
        self.servoMin = [46,46,41,47]
        self.degMin =  [-45,-60,-60,-95]
        self.servoZero = [73,82,75,100]
        self.servoMax = [121,98,123,125]
        self.servoListDegs = [0.0] * 4 # [right, left, base, gripper]
        self.servoListRaw = [self.degToServo(deg,i) for i,deg in enumerate(self.servoListDegs) ]
        self.degMax = [self.ServoToDeg(value, i) for i,value in enumerate(self.servoMax)]
        self.serial = serial.Serial(serialPath, baudrate=baudrate)
        # setup widgets
        self.setUpWidgets()
    
    def closeSerial(self):
        self.serial.close()
    
    def setUpWidgets(self):
        style = {'description_width': 'initial'}
        self.floatDegWidgets = [
            widgets.FloatSlider(value = self.servoListDegs[0], 
                                min = self.degMin[0], max = self.degMax[0], step = 0.1, 
                                description = "Right Servo Degs",
                                continuous_update=False, style=style),
            widgets.FloatSlider(value = self.servoListDegs[1],
                                min = self.degMin[1], max = self.degMax[1], step = 0.1,
                                description = "Left Servo Degs",
                                continuous_update=False, style=style),
            widgets.FloatSlider(value = self.servoListDegs[2], 
                                min = self.degMin[2], max = self.degMax[2], step = 0.1, 
                                description = "Base Servo Degs",
                                continuous_update=False, style=style),
            widgets.FloatSlider(value = self.servoListDegs[3], 
                                min = self.degMin[3], max = self.degMax[3], step = 0.1, 
                                description = "Gripper Servo Degs",
                                continuous_update=False, style=style)
        ]
        for w in self.floatDegWidgets:
            w.observe(self.readAllDegWidgets)
        self.degsTextWidget = widgets.Text(description = "servoListDegs", style=style)
        self.serialCommandTextWidget = widgets.Text(description = "sent command via USB", style=style)
        
        self.rawServoValueWidgets = [
            widgets.IntSlider(value = self.servoListRaw[0],
                              min = self.servoMin[0], max = self.servoMax[0], step = 1, 
                              description = "Right Servo Raw",
                              continuous_update=False, style=style),
            widgets.IntSlider(value = self.servoListRaw[1],
                              min = self.servoMin[1], max = self.servoMax[1], step = 1, 
                              description = "Left Servo Raw",
                              continuous_update=False, style=style),
            widgets.IntSlider(value = self.servoListRaw[2],
                              min = self.servoMin[2], max = self.servoMax[2], step = 1, 
                              description = "Base Servo Raw",
                              continuous_update=False, style=style),
            widgets.IntSlider(value = self.servoListRaw[3],
                              min = self.servoMin[3], max = self.servoMax[3], step = 1, 
                              description = "Gripper Servo Raw",
                              continuous_update=False, style=style)
        ]
        for w in self.rawServoValueWidgets:
            w.observe(self.readAllRawWidgets)
        
        l = [w for w in self.floatDegWidgets]
        l.append(self.degsTextWidget)
        l.append(self.serialCommandTextWidget)
        self.left_box = VBox(l,layout = Layout(width='100%'))
        self.right_box = VBox([w for w in self.rawServoValueWidgets],layout = Layout(width='100%'))
        self.allWidgets = HBox([self.left_box, self.right_box],layout = Layout(width='100%'))
        display(self.allWidgets)
    
    def updateRawValuesFromDegs(self):
        for i in range(4):
            rawValue = self.degToServo(self.servoListDegs[i],i )
            self.servoListRaw[i] = rawValue
            self.rawServoValueWidgets[i].unobserve(self.readAllRawWidgets)
            self.rawServoValueWidgets[i].value = rawValue
            self.rawServoValueWidgets[i].observe(self.readAllRawWidgets)
            
    def updateDegValuesFromRaws(self):
        for i in range(4):
            degValue = self.ServoToDeg(self.servoListRaw[i],i )
            self.servoListDegs[i] = degValue
            self.floatDegWidgets[i].unobserve(self.readAllDegWidgets)
            self.floatDegWidgets[i].value = degValue
            self.floatDegWidgets[i].observe(self.readAllDegWidgets)

    def readAllDegWidgets(self, updateServos = True):
        for i, w in enumerate(self.floatDegWidgets):
            self.servoListDegs[i] = w.value
        if (updateServos):
            self.sendServos()
            self.degsTextWidget.value = str(self.servoListDegs)
        self.updateRawValuesFromDegs()
    
    def readAllRawWidgets(self, updateServos = True):
        for i, w in enumerate(self.rawServoValueWidgets):
            self.servoListRaw[i] = w.value
        if (updateServos):
            self.sendServos(sendRaw = True)
            self.degsTextWidget.value = str(self.servoListDegs)
        self.updateDegValuesFromRaws()
        
    def linearMap (self, X1, Y1, X2, Y2):
        slope = (Y1 - Y2) / (X1 - X2)
        b = Y1 - slope * X1
        return slope, b

    def degToServo(self, deg, servoIdx):
        slope, b = self.linearMap(self.degMin[servoIdx], self.servoMin[servoIdx],
                                  0 ,self.servoZero[servoIdx])
        return int(deg*slope + b)
    
    def ServoToDeg(self, value, servoIdx):
        slope, b = self.linearMap(self.servoMin[servoIdx], self.degMin[servoIdx], 
                                  self.servoZero[servoIdx], 0 )
        return slope * value + b
    
    def valuesToString(self, values):
        if (len(values)!=4):
            return None
        return 's'+','.join([str(v) for v in values]) + 'e'
    
    def sendServos(self, sendRaw = False):
        st = None
        if sendRaw:
            st = self.valuesToString(self.servoListRaw)
        else:
            st = self.valuesToString([self.degToServo(deg,i) for i, deg in enumerate(self.servoListDegs)])
        if (st == None):
            return
        if not self.serial.isOpen():
            self.serial.open()
        self.serial.write(st.encode())
        self.serial.close()
        self.serialCommandTextWidget.value = st

In [6]:
controller = armController()

In [7]:
controller.closeSerial()